<a href="https://colab.research.google.com/github/HiraiG/my_world_model/blob/main/dreamerv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ライブラリのimport

In [ ]:
!pip install setuptools==65.5.0 "wheel<0.40.0"
!pip install gym==0.21.0 gym[atari]==0.21.0 gym[accept-rom-license]==0.21.0 autorom ale-py
!pip install ale-py
!pip install -q -U einops datasets
!pip install wandb
!pip install pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.43.0
    Uninstalling wheel-0.43.0:
      Successfully uninstalled wheel-0.43.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.3 requires setuptools>65.5.1, but you have setuptools 65.5.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 51.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616797 sha256=532b480eb1c3e3aed86a1d5dd12d839cbe50627d208552d9ba4cabf634885336
  Stored in directory: /root/.cache/pip/wheels/81/aa/90/b67df76370d3916a2189b662cf48da38ce41a4e7e58b6abff5
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=b88d57a426a474e808419ae52485db8271b77ea8cc5d47e6be45b05fe39ebcdb
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43f

In [ ]:
import wandb
wandb.login()

# 3d203c250261c0fa5b739811895ac4752a70e3b0

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!nvidia-smi

Sun Mar 31 09:28:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import time
import os
import gc
import random
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import gym
from gym.wrappers import ResizeObservation
import pybullet_envs

import torch
import torch.distributions as td
from torch.distributions import Normal, Categorical, OneHotCategorical, OneHotCategoricalStraightThrough
from torch.distributions.kl import kl_divergence
from torch import nn
from torch.nn import functional as F
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
from einops import rearrange, repeat

import math

## RSSM

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=1000):
        super().__init__()
        self.dim = dim

        # Compute positional encodings
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add positional encodings to input
        return x + self.pe[:, :x.size(1)].detach()


In [ ]:
class RSSM(nn.Module):
    def __init__(self, mlp_hidden_dim: int, rnn_hidden_dim: int, state_dim: int, num_classes: int, actino_dim: int):
        super().__init__()

        self.rnn_hidden_dim = rnn_hidden_dim
        self.state_dim = state_dim
        self.num_classes = num_classes

        # Recurrent model
        # h_t = f(h_t-1, z_t-1, a_t-1)
        self.transition_hidden = nn.Linear(state_dim * num_classes + action_dim, mlp_hidden_dim)
        self.transition = nn.GRUCell(mlp_hidden_dim, rnn_hidden_dim)

        # transition predictor
        self.prior_hidden = nn.Linear(rnn_hidden_dim, mlp_hidden_dim)
        self.prior_logits = nn.Linear(mlp_hidden_dim, state_dim * num_classes)

        # representation model
        self.posterior_hidden = nn.Linear(rnn_hidden_dim + 1536, mlp_hidden_dim)
        self.posterior_logits = nn.Linear(mlp_hidden_dim, state_dim * num_classes)

    def recurrent(self, state: torch.Tensor, action: torch.Tensor, rnn_hidden: torch.Tensor):
        # recullent model: h_t = f(h_t-1, z_t-1, a_t-1)を計算する
        hidden = F.elu(self.transition_hidden(torch.cat([state, action], dim=1)))
        rnn_hidden = self.transition(hidden, rnn_hidden)

        return rnn_hidden  # h_t

    def get_prior(self, rnn_hidden: torch.Tensor, detach: bool = False):
        # transition predictor: \hat{z}_t ~ p(z\hat{z}_t | h_t)
        hidden = F.elu(self.prior_hidden(rnn_hidden))
        logits = self.prior_logits(hidden)
        logits = logits.reshape(logits.shape[0], self.state_dim, self.num_classes)
        if detach:
            logits = logits.detach()

        prior_dist = td.Independent(OneHotCategoricalStraightThrough(logits=logits), 1)
        return prior_dist  # p(z\hat{z}_t | h_t)

    def get_posterior(self, rnn_hidden: torch.Tensor, embedded_obs: torch.Tensor, detach: bool = False):
        # representation predictor: z_t ~ q(z_t | h_t, o_t)
        hidden = F.elu(self.posterior_hidden(torch.cat([rnn_hidden, embedded_obs], dim=1)))
        logits = self.posterior_logits(hidden)
        logits = logits.reshape(logits.shape[0], self.state_dim, self.num_classes)
        if detach:
            logits = logits.detach()

        posterior_dist = td.Independent(OneHotCategoricalStraightThrough(logits=logits), 1)
        return posterior_dist  # q(z_t | h_t, o_t)

In [ ]:
# class RSSM(nn.Module):
#     def __init__(self, mlp_hidden_dim: int, rnn_hidden_dim: int, state_dim: int, num_classes: int, actino_dim: int):
#         super().__init__()

#         self.rnn_hidden_dim = rnn_hidden_dim
#         self.state_dim = state_dim
#         self.num_classes = num_classes

#         # Recurrent model
#         # h_t = f(h_t-1, z_t-1, a_t-1)
#         self.transition_hidden = nn.Linear(state_dim * num_classes + action_dim, mlp_hidden_dim)
#         self.transition = nn.GRUCell(mlp_hidden_dim, rnn_hidden_dim)

#         # transition predictor
#         self.prior_hidden = nn.Linear(rnn_hidden_dim, mlp_hidden_dim)
#         self.prior_logits = nn.Linear(mlp_hidden_dim, state_dim * num_classes)

#         # representation model
#         self.posterior_hidden = nn.Linear(rnn_hidden_dim + 1536, mlp_hidden_dim)
#         self.posterior_logits = nn.Linear(mlp_hidden_dim, state_dim * num_classes)

#     def recurrent(self, state: torch.Tensor, action: torch.Tensor, rnn_hidden: torch.Tensor):
#         # recullent model: h_t = f(h_t-1, z_t-1, a_t-1)を計算する
#         hidden = F.elu(self.transition_hidden(torch.cat([state, action], dim=1)))
#         rnn_hidden = self.transition(hidden, rnn_hidden)

#         return rnn_hidden  # h_t

#     def get_prior(self, rnn_hidden: torch.Tensor, detach: bool = False):
#         # transition predictor: \hat{z}_t ~ p(z\hat{z}_t | h_t)
#         hidden = F.elu(self.prior_hidden(rnn_hidden))
#         logits = self.prior_logits(hidden)
#         logits = logits.reshape(logits.shape[0], self.state_dim, self.num_classes)
#         if detach:
#             logits = logits.detach()

#         prior_dist = td.Independent(OneHotCategoricalStraightThrough(logits=logits), 1)
#         return prior_dist  # p(z\hat{z}_t | h_t)

#     def get_posterior(self, rnn_hidden: torch.Tensor, embedded_obs: torch.Tensor, detach: bool = False):
#         # representation predictor: z_t ~ q(z_t | h_t, o_t)
#         hidden = F.elu(self.posterior_hidden(torch.cat([rnn_hidden, embedded_obs], dim=1)))
#         logits = self.posterior_logits(hidden)
#         logits = logits.reshape(logits.shape[0], self.state_dim, self.num_classes)
#         if detach:
#             logits = logits.detach()

#         posterior_dist = td.Independent(OneHotCategoricalStraightThrough(logits=logits), 1)
#         return posterior_dist  # q(z_t | h_t, o_t)

## エンコーダの実装

### attention

In [ ]:
class LinearAttention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32, num_mem_kv=4):
        super().__init__()

        self.scale = dim_head ** -0.5
        self.heads = heads
        hidden_dim = dim_head * heads

        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)

        self.to_out = nn.Sequential(
            nn.Conv2d(hidden_dim, dim, 1),
            nn.GroupNorm(1, dim),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x ( b, dim, h', w' )
        Returns:
            out ( b, dim, h', w' )
        """
        b, c, h, w = x.shape

        q, k, v = self.to_qkv(x).chunk(3, dim=1)
        q = rearrange(q, "b (h c) x y -> b h c (x y)", h=self.heads)
        k = rearrange(k, "b (h c) x y -> b h c (x y)", h=self.heads)
        v = rearrange(v, "b (h c) x y -> b h c (x y)", h=self.heads)
        # ( b, heads, dim_head, h(=x) * w(=y) )

        q = q.softmax(dim=-2)
        k = k.softmax(dim=-1)

        q = q * self.scale

        context = torch.einsum("b h d n, b h e n -> b h d e", k, v)

        out = torch.einsum("b h d e, b h d n -> b h e n", context, q)
        out = rearrange(out, "b h c (x y) -> b (h c) x y", h=self.heads, x=h, y=w)

        return self.to_out(out)

### encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 48, kernel_size=4, stride=2)
        self.bn1 = nn.BatchNorm2d(48)
        self.conv2 = nn.Conv2d(48, 96, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 192, kernel_size=4, stride=2)
        self.bn3 = nn.BatchNorm2d(192)
        self.conv4 = nn.Conv2d(192, 384, kernel_size=4, stride=2)
        self.bn4 = nn.BatchNorm2d(384)

        self.attention = LinearAttention(384, heads=4, dim_head=32)

    def forward(self, obs: torch.Tensor):
        """
        観測画像をベクトルに埋め込むためのEncoder．

        Parameters
        ----------
        obs : torch.Tensor (B, C, H, W)
            入力となる観測画像．

        Returns
        -------
        embedded_obs : torch.Tensor (B, D)
            観測画像をベクトルに変換したもの．Dは入力画像の幅と高さに依存して変わる．
            入力が(B, 3, 64, 64)の場合，出力は(B, 1536)になる．
        """
        x = F.elu(self.conv1(obs))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = self.bn4(self.conv4(x))
        # print("attention適用前 : ", x.shape)
        embedded_obs = self.attention(x).view(x.size(0), -1)
        # embedded_obs = x.view(x.size(0), -1)
        # print("attention適用後 : ", embedded_obs.shape)

        return embedded_obs  # x_t

## デコーダーの定義

In [ ]:
class Decoder(nn.Module):
    def __init__(self, rnn_hidden_dim: int, state_dim: int, num_classes: int):
        super().__init__()
        self.fc = nn.Linear(state_dim*num_classes + rnn_hidden_dim, 1536)
        self.dc1 = nn.ConvTranspose2d(1536, 192, kernel_size=5, stride=2)
        self.dc2 = nn.ConvTranspose2d(192, 96, kernel_size=5, stride=2)
        self.dc3 = nn.ConvTranspose2d(96, 48, kernel_size=6, stride=2)
        self.dc4 = nn.ConvTranspose2d(48, 1, kernel_size=6, stride=2)


    def forward(self, state: torch.Tensor, rnn_hidden: torch.Tensor):
        """
        決定論的状態と，確率的状態を入力として，観測画像を復元するDecoder．
        出力は多次元正規分布の平均値をとる．

        Paremters
        ---------
        state : torch.Tensor (B, state_dim * num_classes)
            確率的状態．
        rnn_hidden : torch.Tensor (B, rnn_hidden_dim)
            決定論的状態．

        Returns
        -------
        obs_dist : torch.distribution.Independent
            観測画像を再構成するための多次元正規分布．
        """
        hidden = self.fc(torch.cat([state, rnn_hidden], dim=1))
        hidden = hidden.view(hidden.size(0), 1536, 1, 1)
        hidden = F.elu(self.dc1(hidden))
        hidden = F.elu(self.dc2(hidden))
        hidden = F.elu(self.dc3(hidden))
        mean = self.dc4(hidden)

        obs_dist = td.Independent(td.Normal(mean, 1), 3)
        return obs_dist  # p(\hat{x}_t | h_t, z_t)

In [ ]:
# class Encoder(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.conv1 = nn.Conv2d(1, 48, kernel_size=4, stride=2)
#         self.conv2 = nn.Conv2d(48, 96, kernel_size=4, stride=2)
#         self.conv3 = nn.Conv2d(96, 192, kernel_size=4, stride=2)
#         self.conv4 = nn.Conv2d(192, 384, kernel_size=4, stride=2)

#     def forward(self, obs: torch.Tensor):
#         """
#         観測画像をベクトルに埋め込むためのEncoder．

#         Parameters
#         ----------
#         obs : torch.Tensor (B, C, H, W)
#             入力となる観測画像．

#         Returns
#         -------
#         embedded_obs : torch.Tensor (B, D)
#             観測画像をベクトルに変換したもの．Dは入力画像の幅と高さに依存して変わる．
#             入力が(B, 3, 64, 64)の場合，出力は(B, 1536)になる．
#         """
#         hidden = F.elu(self.conv1(obs))
#         hidden = F.elu(self.conv2(hidden))
#         hidden = F.elu(self.conv3(hidden))
#         embedded_obs = self.conv4(hidden).reshape(hidden.size(0), -1)

#         return embedded_obs  # x_t

# class Decoder(nn.Module):
#     def __init__(self, rnn_hidden_dim: int, state_dim: int, num_classes: int):
#         super().__init__()
#         self.fc = nn.Linear(state_dim*num_classes + rnn_hidden_dim, 1536)
#         self.dc1 = nn.ConvTranspose2d(1536, 192, kernel_size=5, stride=2)
#         self.dc2 = nn.ConvTranspose2d(192, 96, kernel_size=5, stride=2)
#         self.dc3 = nn.ConvTranspose2d(96, 48, kernel_size=6, stride=2)
#         self.dc4 = nn.ConvTranspose2d(48, 1, kernel_size=6, stride=2)


#     def forward(self, state: torch.Tensor, rnn_hidden: torch.Tensor):
#         """
#         決定論的状態と，確率的状態を入力として，観測画像を復元するDecoder．
#         出力は多次元正規分布の平均値をとる．

#         Paremters
#         ---------
#         state : torch.Tensor (B, state_dim * num_classes)
#             確率的状態．
#         rnn_hidden : torch.Tensor (B, rnn_hidden_dim)
#             決定論的状態．

#         Returns
#         -------
#         obs_dist : torch.distribution.Independent
#             観測画像を再構成するための多次元正規分布．
#         """
#         hidden = self.fc(torch.cat([state, rnn_hidden], dim=1))
#         hidden = hidden.view(hidden.size(0), 1536, 1, 1)
#         hidden = F.elu(self.dc1(hidden))
#         hidden = F.elu(self.dc2(hidden))
#         hidden = F.elu(self.dc3(hidden))
#         mean = self.dc4(hidden)

#         obs_dist = td.Independent(td.Normal(mean, 1), 3)
#         return obs_dist  # p(\hat{x}_t | h_t, z_t)

## 報酬関数

In [ ]:
class RewardModel(nn.Module):
    def __init__(self, hidden_dim: int, rnn_hidden_dim: int, state_dim: int, num_classes: int):
        super().__init__()
        self.fc1 = nn.Linear(state_dim*num_classes + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)

    def forward(self, state: torch.Tensor, rnn_hidden: torch.Tensor):
        """
        決定論的状態と，確率的状態を入力として，報酬を予測するモデル．
        出力は正規分布の平均値をとる．

        Paremters
        ---------
        state : torch.Tensor (B, state_dim * num_classes)
            確率的状態．
        rnn_hidden : torch.Tensor (B, rnn_hidden_dim)
            決定論的状態．

        Returns
        -------
        reward_dist : torch.distribution.Independent
            報酬を予測するための正規分布．
        """
        hidden = F.elu(self.fc1(torch.cat([state, rnn_hidden], dim=1)))
        hidden = F.elu(self.fc2(hidden))
        hidden = F.elu(self.fc3(hidden))
        mean = self.fc4(hidden)

        reward_dist = td.Independent(td.Normal(mean, 1),  1)
        return reward_dist  # p(\hat{r}_t | h_t, z_t)

In [ ]:
class DiscountModel(nn.Module):
    def __init__(self, hidden_dim: int, rnn_hidden_dim: int, state_dim: int, num_classes: int):
        super().__init__()
        self.fc1 = nn.Linear(state_dim*num_classes + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)

    def forward(self, state: torch.Tensor, rnn_hidden: torch.Tensor):
        """
        決定論的状態と，確率的状態を入力として，現在の状態がエピソード終端かどうか判別するモデル．
        出力はベルヌーイ分布の平均値をとる．

        Paremters
        ---------
        state : torch.Tensor (B, state_dim * num_classes)
            確率的状態．
        rnn_hidden : torch.Tensor (B, rnn_hidden_dim)
            決定論的状態．

        Returns
        -------
        discount_dist : torch.distribution.Independent
            状態が終端かどうかを予測するためのベルヌーイ分布．
        """
        hidden = F.elu(self.fc1(torch.cat([state, rnn_hidden], dim=1)))
        hidden = F.elu(self.fc2(hidden))
        hidden = F.elu(self.fc3(hidden))
        mean= self.fc4(hidden)

        discount_dist = td.Independent(td.Bernoulli(logits=mean),  1)
        return discount_dist  # p(\hat{\gamma}_t | h_t, z_t)

## Actor-Critic

In [ ]:
class Actor(nn.Module):
    def __init__(self, action_dim: int, hidden_dim: int, rnn_hidden_dim: int, state_dim: int, num_classes: int):
        super().__init__()

        self.fc1 = nn.Linear(state_dim * num_classes + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, action_dim)

    def forward(self, state: torch.tensor, rnn_hidden: torch.Tensor, eval: bool = False):
        """
        確率的状態を入力として，criticで推定される価値が最大となる行動を出力する．

        Parameters
        ----------
        state : torch.Tensor (B, state_dim * num_classes)
            確率的状態．
        rnn_hidden : torch.Tensor (B, rnn_hidden_dim)
            決定論的状態．

        Returns
        -------
        action : torch.Tensor (B, 1)
            行動．
        action_log_prob : torch.Tensor(B, 1)
            予測した行動をとる確率の対数．
        action_entropy : torch.Tensor(B, 1)
            予測した確率分布のエントロピー．エントロピー正則化に使用．
        """
        hidden = F.elu(self.fc1(torch.cat([state, rnn_hidden], dim=1)))
        hidden = F.elu(self.fc2(hidden))
        hidden = F.elu(self.fc3(hidden))
        hidden = F.elu(self.fc4(hidden))
        logits = self.out(hidden)

        if eval:
            action = torch.argmax(logits, dim=1)
            action = F.one_hot(action, logits.shape[1])
            return action, None, None

        action_dist = OneHotCategorical(logits=logits)  # 行動をサンプリングする分布: p_{\psi} (\hat{a}_t | \hat{z}_t)
        action = action_dist.sample()  # 行動: \hat{a}_t

        # Straight-Throught Estimatorで勾配を通す．
        action = action + (action_dist.probs - action_dist.probs.detach())

        action_log_prob = action_dist.log_prob(torch.round(action.detach()))
        action_entropy = action_dist.entropy()

        return action, action_log_prob, action_entropy

In [ ]:
class Critic(nn.Module):
    def __init__(self, hidden_dim: int, rnn_hidden_dim: int, state_dim: int, num_classes: int):
        super().__init__()

        self.fc1 = nn.Linear(state_dim * num_classes + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, state: torch.tensor, rnn_hidden: torch.Tensor):
        """
        確率的状態を入力として，価値関数(lambda target)の値を予測する．．

        Parameters
        ----------
        state : torch.Tensor (B, state_dim * num_classes)
            確率的状態．
        rnn_hidden : torch.Tensor (B, rnn_hidden_dim)
            決定論的状態．

        Returns
        -------
        value : torch.Tensor (B, 1)
            入力された状態に対する状態価値関数の予測値．
        """
        hidden = F.elu(self.fc1(torch.cat([state, rnn_hidden], dim=1)))
        hidden = F.elu(self.fc2(hidden))
        hidden = F.elu(self.fc3(hidden))
        hidden = F.elu(self.fc4(hidden))
        value = self.out(hidden)

        return value  # v_{\xi}(\hat{z}_t)の近似（論文に合わせて決定論的）

## リプレイバッファ

In [ ]:
class ReplayBuffer(object):
    """
    RNNを用いて訓練するのに適したリプレイバッファ
    """
    def __init__(self, capacity, observation_shape, action_dim):
        self.capacity = capacity

        self.observations = np.zeros((capacity, *observation_shape), dtype=np.float32)
        self.actions = np.zeros((capacity, action_dim), dtype=np.float32)
        self.rewards = np.zeros((capacity, 1), dtype=np.float32)
        self.done = np.zeros((capacity, 1), dtype=bool)

        self.index = 0
        self.is_filled = False

    def push(self, observation, action, reward, done):
        """
        リプレイバッファに経験を追加する
        """
        self.observations[self.index] = observation
        self.actions[self.index] = action
        self.rewards[self.index] = reward
        self.done[self.index] = done

        # indexは巡回し, 最も古い経験を上書きする
        if self.index == self.capacity - 1:
            self.is_filled = True
        self.index = (self.index + 1) % self.capacity

    def sample(self, batch_size, chunk_length):
        """
        経験をリプレイバッファからサンプルします. （ほぼ）一様なサンプルです
        結果として返ってくるのは観測(画像), 行動, 報酬, 終了シグナルについての(batch_size, chunk_length, 各要素の次元)の配列です
        各バッチは連続した経験になっています
        注意: chunk_lengthをあまり大きな値にすると問題が発生する場合があります
        """
        episode_borders = np.where(self.done)[0]
        sampled_indexes = []
        for _ in range(batch_size):
            cross_border = True
            while cross_border:
                initial_index = np.random.randint(len(self) - chunk_length + 1)
                final_index = initial_index + chunk_length - 1
                cross_border = np.logical_and(initial_index <= episode_borders,
                                              episode_borders < final_index).any()#論理積
            sampled_indexes += list(range(initial_index, final_index + 1))

        sampled_observations = self.observations[sampled_indexes].reshape(
            batch_size, chunk_length, *self.observations.shape[1:])
        sampled_actions = self.actions[sampled_indexes].reshape(
            batch_size, chunk_length, self.actions.shape[1])
        sampled_rewards = self.rewards[sampled_indexes].reshape(
            batch_size, chunk_length, 1)
        sampled_done = self.done[sampled_indexes].reshape(
            batch_size, chunk_length, 1)
        return sampled_observations, sampled_actions, sampled_rewards, sampled_done

    def __len__(self):
        return self.capacity if self.is_filled else self.index

## ターゲット関数の計算

In [ ]:
def calculate_lambda_target(rewards: torch.Tensor, discounts: torch.Tensor, values: torch.Tensor, lambda_: float):
    """
    lambda targetを計算する関数．

    Parameters
    ---------
    rewards : torch.Tensor (imagination_horizon, D)
        報酬．1次元目が時刻tを表しており，2次元目は自由な次元数にでき，想像の軌道を作成するときに入力されるサンプルindexと考える．
    discounts : torch.Tensor (imagination_horizon, D)
        割引率．gammaそのままを利用するのではなく，DiscountModelの出力をかけて利用する．
    values : torch.Tensor (imagination_horizon, D)
        状態価値関数．criticで予測された値を利用するが，Dreamer v2ではtarget networkで計算する．
    lambda_ : float
        lambda targetのハイパラ．

    Returns
    -------
    V_lambda : torch.Tensor (imagination_horizon, D)
        lambda targetの値．
    """
    V_lambda = torch.zeros_like(rewards)

    for t in reversed(range(rewards.shape[0])):
        if t == rewards.shape[0] - 1:
            V_lambda[t] = rewards[t] + discounts[t] * values[t]  # t=Hの場合（式4の下の条件）
        else:
            V_lambda[t] = rewards[t] + discounts[t] * ((1-lambda_) * values[t+1] + lambda_ * V_lambda[t+1])

    return V_lambda

In [ ]:
def preprocess_obs(obs):
    """
    画像の変換. [0, 255] -> [0, 1]
    """
    height, width = obs.shape[0], obs.shape[1]
    obs = Image.fromarray(obs)
    obs = obs.convert("L")
    obs = np.array(obs).reshape(height, width, 1)
    obs = obs.astype(np.float32)
    normalized_obs = obs / 255.0 - 0.5
    return normalized_obs

In [ ]:
class Agent:
    """
    ActionModelに基づき行動を決定する. そのためにRSSMを用いて状態表現をリアルタイムで推論して維持するクラス
    """
    def __init__(self, encoder, rssm, action_model):
        self.encoder = encoder
        self.rssm = rssm
        self.action_model = action_model

        self.device = next(self.action_model.parameters()).device
        self.rnn_hidden = torch.zeros(1, rssm.rnn_hidden_dim, device=self.device)

    def __call__(self, obs, eval=False):
        # preprocessを適用, PyTorchのためにChannel-Firstに変換
        obs = preprocess_obs(obs)
        obs = torch.as_tensor(obs, device=self.device)
        obs = obs.transpose(1, 2).transpose(0, 1).unsqueeze(0)

        with torch.no_grad():
            # 観測を低次元の表現に変換し, posteriorからのサンプルをActionModelに入力して行動を決定する
            embedded_obs = self.encoder(obs)
            state_posterior = self.rssm.get_posterior(self.rnn_hidden, embedded_obs)
            state = state_posterior.sample().flatten(1)
            action, _, _  = self.action_model(state, self.rnn_hidden, eval=eval)

            # 次のステップのためにRNNの隠れ状態を更新しておく
            self.rnn_hidden = self.rssm.recurrent(state, action, self.rnn_hidden)

        return action.squeeze().cpu().numpy()

    #RNNの隠れ状態をリセット
    def reset(self):
        self.rnn_hidden = torch.zeros(1, self.rssm.rnn_hidden_dim, device=self.device)

## 学習の準備

In [ ]:
class Config:
    def __init__(self, **kwargs):
        # data settings
        self.buffer_size = 2_000_000  # バッファにためるデータの上限
        self.batch_size = 50  # 学習時のバッチサイズ
        self.seq_length = 50  # 各バッチの系列長
        self.imagination_horizon = 15  # 想像上の軌道の系列長

        # model dimensions
        self.state_dim = 32  # 確率的な状態の次元数
        self.num_classes = 32  # 確率的な状態のクラス数（離散表現のため）
        self.rnn_hidden_dim = 600  # 決定論的な状態の次元数
        self.mlp_hidden_dim = 400  # MLPの隠れ層の次元数

        # learning params
        self.model_lr = 2e-4  # world model(trainsition / prior / posterior / discount / image predictor)の学習率
        self.actor_lr = 4e-5  # actorの学習率
        self.critic_lr = 1e-4  # criticの学習率
        self.epsilon = 1e-5  # optimizerのepsilonの値
        self.weight_decay = 1e-6  # weight decayの係数
        self.gradient_clipping = 100  # 勾配クリッピング
        self.kl_scale = 0.1  # kl lossのスケーリング係数
        self.kl_balance = 0.8  # kl balancingの係数(fix posterior)
        self.actor_entropy_scale = 1e-3  # entropy正則化のスケーリング係数
        self.slow_critic_update = 100  # target critic networkの更新頻度
        self.reward_loss_scale = 1.0  # reward lossのスケーリング係数
        self.discount_loss_scale = 5.0  # discount lossのスケーリング係数

        # lambda return params
        self.discount = 0.99  # 割引率
        self.lambda_ = 0.95  # lambda returnのパラメータ

        # learning piriod settings
        self.num_iter = 10_000_000  # イテレーション数
        self.seed_iter = 50_000  # 事前にランダム行動で探索する回数
        self.update_freq = 4  # パラメータを更新する頻度
        self.eval_freq = 10  # 評価頻度（エピソード）
        self.eval_episodes = 5  # 評価に用いるエピソード数

cfg = Config()

In [ ]:
class RepeatAction(gym.Wrapper):
    """
    同じ行動を指定された回数自動的に繰り返すラッパー. 観測は最後の行動に対応するものになる
    """
    def __init__(self, env, skip=4):
        gym.Wrapper.__init__(self, env)
        self.height = env.observation_space.shape[0]
        self.width = env.observation_space.shape[1]
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        total_reward = 0.0
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)

            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

### 環境

In [ ]:
def make_env(seed=1234, img_size=64):
    env = gym.make("ALE/MontezumaRevenge-v5")
    # env = gym.make('HalfCheetahBulletEnv-v0')

    # シード固定
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    # np.random.seed(seed)

    # Dreamerでは観測は64x64のRGB画像
    env = ResizeObservation(env, (img_size, img_size))
    env = RepeatAction(env, skip=4)

    return env

In [ ]:
def set_seed(seed: int) -> None:
    """
    Pytorch, NumPyのシード値を固定します．これによりモデル学習の再現性を担保できます．

    Parameters
    ----------
    seed : int
        シード値．
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# モデルパラメータをGoogleDriveに保存・後で読み込みするためのヘルパークラス
class TrainedModels:
    def __init__(self, *models) -> None:
        """
        コンストラクタ．

        Parameters
        ----------
        models : nn.Module
            保存するモデル．複数モデルを渡すことができます．

        使用例: trained_models = TraindModels(encoder, rssm, value_model, action_model)
        """
        assert np.all([nn.Module in model.__class__.__bases__ for model in models]), "Arguments for TrainedModels need to be nn models."

        self.models = models

    def save(self, dir: str) -> None:
        """
        initで渡したモデルのパラメータを保存します．
        パラメータのファイル名は01.pt, 02.pt, ... のように連番になっています．

        Parameters
        ----------
        dir : str
            パラメータの保存先．
        """
        for i, model in enumerate(self.models):
            torch.save(
                model.state_dict(),
                os.path.join(dir, f"{str(i + 1).zfill(2)}.pt")
            )

    def load(self, dir: str, device: str) -> None:
        """
        initで渡したモデルのパラメータを読み込みます．

        Parameters
        ----------
        dir : str
            パラメータの保存先．
        device : str
            モデルをどのデバイス(CPU or GPU)に載せるかの設定．
        """
        for i, model in enumerate(self.models):
            model.load_state_dict(
                torch.load(
                    os.path.join(dir, f"{str(i + 1).zfill(2)}.pt"),
                    map_location=device
                )
            )

In [ ]:
def evaluation(env: RepeatAction, policy: Agent, step: int, cfg: Config):
    """
    評価用の関数．

    Parameters
    ----------
    env: RepeatAction
        環境のインスタンス．
    policy : Agent
        エージェントのインスタンス．
    step : int
        現状の訓練のステップ数．
    cfg : Config
        コンフィグ．

    Returns
    -------
    max_ep_rewards : float
        評価中に1エピソードで得た最大の報酬和．
    """
    all_ep_rewards = []

    with torch.no_grad():
        for i in range(cfg.eval_episodes):
            obs = env.reset()  # 環境をリセット
            policy.reset()  # RNNの隠れ状態をリセット
            done = False  # 終端条件
            episode_reward = 0  # エピソードでの報酬和
            while not done:
                action = policy(obs, eval=True)
                action_int = np.argmax(action)

                obs, reward, done, _ = env.step(action_int)
                episode_reward += reward

            all_ep_rewards.append(episode_reward)

        mean_ep_rewards = np.mean(all_ep_rewards)
        max_ep_rewards = np.max(all_ep_rewards)
        print(f"Eval(iter={step}) mean: {mean_ep_rewards:.4f} max: {max_ep_rewards:.4f}")

    return max_ep_rewards

## 学習

In [ ]:
# モデル等の初期化
set_seed(1234)
env = make_env()
device = "cuda" if torch.cuda.is_available() else "cpu"

action_dim = env.action_space.n
# action_dim = env.action_space.shape[0]
# リプレイバッファ
replay_buffer = ReplayBuffer(
    capacity=cfg.buffer_size,
    observation_shape=(64, 64, 1),
    action_dim=env.action_space.n
    # action_dim = env.action_space.shape[0]
)

# モデル
rssm = RSSM(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes, action_dim).to(device)
encoder = Encoder().to(device)
decoder = Decoder(cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
reward_model =  RewardModel(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
discount_model = DiscountModel(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
actor = Actor(action_dim, cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
critic = Critic(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
target_critic = Critic(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
target_critic.load_state_dict(critic.state_dict())

trained_models = TrainedModels(
    rssm,
    encoder,
    decoder,
    reward_model,
    discount_model,
    actor,
    critic
)

# optimizer
wm_params = (list(rssm.parameters())         +
             list(encoder.parameters())      +
             list(decoder.parameters())      +
             list(reward_model.parameters()) +
             list(discount_model.parameters())
            )
wm_optimizer = torch.optim.AdamW(wm_params, lr=cfg.model_lr, eps=cfg.epsilon, weight_decay=cfg.weight_decay)
actor_optimizer = torch.optim.AdamW(actor.parameters(), lr=cfg.actor_lr, eps=cfg.epsilon, weight_decay=cfg.weight_decay)
critic_optimizer = torch.optim.AdamW(critic.parameters(), lr=cfg.critic_lr, eps=cfg.epsilon, weight_decay=cfg.weight_decay)

In [ ]:
# ランダム行動でバッファを埋める

# pbar = tqdm(total=cfg.seed_iter, unit='buffer size')
while len(replay_buffer) < cfg.seed_iter:
    obs = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_obs, reward, done, _ = env.step(action)
        action = F.one_hot(torch.tensor(action), num_classes=env.action_space.n)
        # action = F.one_hot(torch.tensor(action), num_classes=env.action_space.shape[0])
        replay_buffer.push(preprocess_obs(obs), action, np.tanh(reward), done)
        obs = next_obs
    # pbar.update(len(replay_buffer))

# episodes = 5
# for episode in tqdm(range(episodes), total = episodes):
#     obs = env.reset()
#     done = False
#     while not done:
#         action = env.action_space.sample()
#         next_obs, reward, done, _ = env.step(action)
#         replay_buffer.push(obs, action, reward, done)
#         obs = next_obs

# episodes = 5
# for episode in tqdm(range(episodes), total=episodes):
#     obs = env.reset()
#     done = False
#     while not done:
#         action = env.action_space.sample()  # ランダムな行動をサンプリング
#         next_obs, reward, done, _ = env.step(action)
#         # 行動をone-hotベクトルに変換
#         action = int((action - env.action_space.low) / (env.action_space.high - env.action_space.low) * (env.action_space.shape[0] - 1))
#         replay_buffer.push(preprocess_obs(obs), action, np.tanh(reward), done)  # 前処理した観測、行動、報酬、終了フラグをバッファに追加
#         obs = next_obs
# pbar.close()

### 学習の開始

In [ ]:
# 学習を行う
# 環境と相互作用 → 一定イテレーションでモデル更新を繰り返す
policy = Agent(encoder, rssm, actor)

# 環境，収益等の初期化
obs = env.reset()
done = False
total_reward = 0
total_episode = 1
best_reward = 0

# wandbの初期化
wandb.init(project="強化学習_最終課題")
wandb.config.update(cfg)

for iteration in tqdm(range(cfg.num_iter)):
    with torch.no_grad():
        # 環境と相互作用
        action = policy(obs)  # モデルで行動をサンプリング(one-hot)
        action_int = np.argmax(action)  # 環境に渡すときはint型
        next_obs, reward, done, _ = env.step(action_int)  # 環境を進める

        # 得たデータをリプレイバッファに追加して更新
        replay_buffer.push(preprocess_obs(obs), action, np.tanh(reward), done)  # x_t, a_t, r_t, gamma_t
        obs = next_obs
        total_reward += reward

    if (iteration + 1) % cfg.update_freq == 0:
        # モデルの学習
        # リプレイバッファからデータをサンプリングする
        # (batch size, seq_lenght, *data shape)
        observations, actions, rewards, done_flags =\
            replay_buffer.sample(cfg.batch_size, cfg.seq_length)
        done_flags = 1 - done_flags  # 終端でない場合に1をとる

        # torchで扱える形（seq lengthを最初の次元に，画像はchnnelを最初の次元にする）に変形，observationの前処理
        observations = torch.permute(torch.as_tensor(observations, device=device), (1, 0, 4, 2, 3))  # (T, B, C, H, W)
        actions = torch.as_tensor(actions, device=device).transpose(0, 1)  # (T, B, action dim)
        rewards = torch.as_tensor(rewards, device=device).transpose(0, 1)  # (T, B, 1)
        done_flags = torch.as_tensor(done_flags, device=device).transpose(0, 1).float()  # (T, B, 1)

        # =================
        # world modelの学習
        # =================
        # 観測をベクトルに埋めこみ
        emb_observations = encoder(observations.reshape(-1, 1, 64, 64)).view(cfg.seq_length, cfg.batch_size, -1)  # (T, B, 1536)

        # 状態表現z，行動aはゼロで初期化
        # バッファから取り出したデータをt={1, ..., seq length}とするなら，以下はz_1とみなせる
        state = torch.zeros(cfg.batch_size, cfg.state_dim*cfg.num_classes, device=device)
        rnn_hidden = torch.zeros(cfg.batch_size, cfg.rnn_hidden_dim, device=device)

        # 各観測に対して状態表現を計算
        # タイムステップごとに計算するため，先に格納するTensorを定義する(t={1, ..., seq length})
        states = torch.zeros(cfg.seq_length, cfg.batch_size, cfg.state_dim*cfg.num_classes, device=device)
        rnn_hiddens = torch.zeros(cfg.seq_length, cfg.batch_size, cfg.rnn_hidden_dim, device=device)

        # prior, posteriorを計算してKL lossを計算する
        kl_loss = 0
        for i in range(cfg.seq_length-1):
            # rnn hiddenを更新
            rnn_hidden = rssm.recurrent(state, actions[i], rnn_hidden)  # h_t+1

            # prior, posteriorを計算
            next_state_prior = rssm.get_prior(rnn_hidden) # \hat{z}_t+1
            next_state_posterior = rssm.get_posterior(rnn_hidden, emb_observations[i+1])  # z_t+1

            # posteriorからzをサンプリング
            state = next_state_posterior.rsample().flatten(1)
            rnn_hiddens[i+1] = rnn_hidden  # h_t+1
            states[i+1] = state  # z_t+1

            # KL lossを計算
            kl_loss += (
                        (cfg.kl_balance * kl_divergence(rssm.get_posterior(rnn_hidden, emb_observations[i+1], detach=True), next_state_prior) + \
                        (1 - cfg.kl_balance) * kl_divergence(next_state_posterior, rssm.get_prior(rnn_hidden, detach=True)))
                       ).mean()
        kl_loss /= (cfg.seq_length - 1)

        # 初期状態は使わない
        rnn_hiddens = rnn_hiddens[1:]  # (seq lenghth - 1, batch size rnn hidden)
        states = states[1:]  # (seq length - 1, batch size, state dim * num_classes)

        # 得られた状態を利用して再構成，報酬，終端フラグを予測
        # そのままでは時間方向，バッチ方向で次元が多いため平坦化
        flatten_rnn_hiddens = rnn_hiddens.view(-1, cfg.rnn_hidden_dim)  # ((T-1) * B, rnn hidden)
        flatten_states = states.view(-1, cfg.state_dim * cfg.num_classes)  # ((T-1) * B, state_dim * num_classes)

        # 上から再構成，報酬，終端フラグ予測
        obs_dist = decoder(flatten_states, flatten_rnn_hiddens)  # (T * B, 3, 64, 64)
        reward_dist = reward_model(flatten_states, flatten_rnn_hiddens)  # (T * B, 1)
        discount_dist = discount_model(flatten_states, flatten_rnn_hiddens)  # (T * B, 1)

        # 各予測に対する損失の計算（対数尤度）
        C, H, W = observations.shape[2:]
        obs_loss = -obs_dist.log_prob(observations[1:].reshape(-1, C, H, W)).mean()
        reward_loss = -reward_dist.log_prob(rewards[:-1].reshape(-1, 1)).mean()
        discount_loss = -discount_dist.log_prob(done_flags[:-1].reshape(-1, 1)).mean()

        # 総和をとってモデルを更新
        wm_loss = obs_loss + cfg.reward_loss_scale * reward_loss + cfg.discount_loss_scale * discount_loss + cfg.kl_scale * kl_loss

        wm_optimizer.zero_grad()
        wm_loss.backward()
        clip_grad_norm_(wm_params, cfg.gradient_clipping)
        wm_optimizer.step()

        #====================
        # Actor, Criticの更新
        #===================
        # wmから得た状態の勾配を切っておく
        flatten_rnn_hiddens = flatten_rnn_hiddens.detach()
        flatten_states = flatten_states.detach()

        # priorを用いた状態予測
        # 格納する空のTensorを用意
        imagined_states = torch.zeros(cfg.imagination_horizon + 1,
                                      *flatten_states.shape,
                                      device=flatten_states.device)
        imagined_rnn_hiddens = torch.zeros(cfg.imagination_horizon + 1,
                                           *flatten_rnn_hiddens.shape,
                                           device=flatten_rnn_hiddens.device)
        imagined_action_log_probs = torch.zeros((cfg.imagination_horizon + 1, cfg.batch_size * (cfg.seq_length-1)),
                                                device=flatten_rnn_hiddens.device)
        imagined_action_entropys = torch.zeros((cfg.imagination_horizon + 1, cfg.batch_size * (cfg.seq_length-1)),
                                                device=flatten_rnn_hiddens.device)

        #　未来予測をして想像上の軌道を作る前に, 最初の状態としては先ほどモデルの更新で使っていた
        # リプレイバッファからサンプルされた観測データを取り込んだ上で推論した状態表現を使う
        imagined_states[0] = flatten_states
        imagined_rnn_hiddens[0] = flatten_rnn_hiddens

        # open-loopで予測
        for i in range(1, cfg.imagination_horizon + 1):
            actions, action_log_probs, action_entropys = actor(flatten_states, flatten_rnn_hiddens)  # ((T-1) * B, action dim)

            # rnn hiddenを更新, priorで次の状態を予測
            rnn_hidden = rssm.recurrent(flatten_states, actions, flatten_rnn_hiddens)  # h_t+1
            flatten_states_prior = rssm.get_prior(flatten_rnn_hiddens)
            flatten_states = flatten_states_prior.rsample().flatten(1)

            imagined_rnn_hiddens[i] = flatten_rnn_hiddens
            imagined_states[i] = flatten_states
            imagined_action_log_probs[i-1] = action_log_probs
            imagined_action_entropys[i-1] = action_entropys

        actions, action_log_probs, action_entropys = actor(flatten_states, flatten_rnn_hiddens)  # ((T-1) * B, action dim)
        imagined_action_log_probs[-1] = action_log_probs
        imagined_action_entropys[-1] = action_entropys

        # 得られた状態から報酬を予測
        flatten_imagined_states = imagined_states.view(-1, cfg.state_dim * cfg.num_classes)  # ((imagination horizon + 1) * (T-1) * B, state dim * num classes)
        flatten_imagined_rnn_hiddens = imagined_rnn_hiddens.view(-1, cfg.rnn_hidden_dim)  # ((imagination horizon + 1) * (T-1) * B, rnn hidden)

        # reward, done_flagsは分布なので平均値をとる
        # ((imagination horizon + 1), (T-1) * B)
        imagined_rewards = reward_model(flatten_imagined_states, flatten_imagined_rnn_hiddens).mean.view(cfg.imagination_horizon + 1, -1)
        imagined_values = critic(flatten_imagined_states, flatten_imagined_rnn_hiddens).view(cfg.imagination_horizon + 1, -1)
        target_values = target_critic(flatten_imagined_states, flatten_imagined_rnn_hiddens).view(cfg.imagination_horizon + 1, -1)
        imagined_done_flags = discount_model(flatten_imagined_states, flatten_imagined_rnn_hiddens).base_dist.probs.view(cfg.imagination_horizon + 1, -1)

        # lambda targetの計算
        discount_arr = cfg.discount * torch.round(imagined_done_flags)
        lambda_target = calculate_lambda_target(imagined_rewards, discount_arr, target_values, cfg.lambda_)

        # criticの損失を計算
        critic_loss = (0.5 * (imagined_values - lambda_target.detach()) ** 2).mean()
        critic_optimizer.zero_grad()
        critic_loss.backward(retain_graph=True)
        clip_grad_norm_(critic.parameters(), cfg.gradient_clipping)
        critic_optimizer.step()

        # actorの損失を計算
        actor_loss = -(imagined_action_log_probs * (lambda_target - target_values).detach() + cfg.actor_entropy_scale * imagined_action_entropys).mean()

        actor_optimizer.zero_grad()
        actor_loss.backward()
        clip_grad_norm_(actor.parameters(), cfg.gradient_clipping)
        actor_optimizer.step()

        # wandbに記録する損失関数
        wandb.log({"kl_loss": kl_loss.item(),
                "obs_loss": obs_loss.item(),
                "reward_loss": reward_loss.item(),
                "discount_loss": discount_loss.item(),
                "critic_loss": critic_loss.item(),
                "actor_loss": actor_loss.item()})

    if (iteration + 1) % cfg.slow_critic_update == 0:
        target_critic.load_state_dict(critic.state_dict())

    # エピソードが終了した時に再初期化
    if done:
        print(f"episode: {total_episode} total_reward: {total_reward:.8f}")
        print(f"num iter: {iteration} kl loss: {kl_loss.item():.8f} obs loss: {obs_loss.item():.8f} "
              f"rewrd loss: {reward_loss.item():.8f} discount loss: {discount_loss.item():.8f} "
              f"critic loss: {critic_loss.item():.8f} actor loss: {actor_loss.item():.8f}"
        )

        obs = env.reset()
        done = False
        total_reward = 0
        total_episode += 1

        # wandb
        wandb.log({"episode_reward": total_reward})

        policy.reset()

        # 一定エピソードごとに評価
        if total_episode % cfg.eval_freq == 0:
            eval_reward = evaluation(env, policy, iteration, cfg)
            if eval_reward > best_reward:
                best_reward = eval_reward
                trained_models.save("./")

            # wandb
            wandb.log({"eval_reward": eval_reward})

        obs = env.reset()
        policy.reset()

wandb.finish()

wandb: Currently logged in as: slhs1328. Use `wandb login --relogin` to force relogin


  0%|          | 0/10000000 [00:00<?, ?it/s]

episode: 1 total_reward: 0.00000000
num iter: 44 kl loss: 4.58907318 obs loss: 3868.78125000 rewrd loss: 0.91906548 discount loss: 0.37811649 critic loss: 0.00299728 actor loss: -0.10519888
episode: 2 total_reward: 0.00000000
num iter: 138 kl loss: 0.99502927 obs loss: 3780.68334961 rewrd loss: 0.91897970 discount loss: 0.00005927 critic loss: 0.00029442 actor loss: 0.01883146
episode: 3 total_reward: 0.00000000
num iter: 187 kl loss: 0.34717867 obs loss: 3771.55419922 rewrd loss: 0.91895527 discount loss: 0.00001332 critic loss: 0.00017597 actor loss: 0.00514786
episode: 4 total_reward: 0.00000000
num iter: 253 kl loss: 0.09160181 obs loss: 3768.23071289 rewrd loss: 0.91894650 discount loss: 0.00000959 critic loss: 0.00008528 actor loss: 0.00693887
episode: 5 total_reward: 0.00000000
num iter: 332 kl loss: 0.01621269 obs loss: 3767.06811523 rewrd loss: 0.91894311 discount loss: 0.00000846 critic loss: 0.00004523 actor loss: -0.00997031
episode: 6 total_reward: 0.00000000
num iter: 375

KeyboardInterrupt: 

In [ ]:
# モデルの保存
from google.colab import drive
drive.mount('/content/drive')

trained_models.save("drive/MyDrive/深層強化学習/最終課題/model")

In [ ]:
# 環境の読み込み
env = make_env()
device = "cuda" if torch.cuda.is_available() else "cpu"

# 学習済みモデルの読み込み
rssm = RSSM(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes, action_dim).to(device)
encoder = Encoder().to(device)
decoder = Decoder(cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
reward_model =  RewardModel(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
discount_model = DiscountModel(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
actor = Actor(action_dim, cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)
critic = Critic(cfg.mlp_hidden_dim, cfg.rnn_hidden_dim, cfg.state_dim, cfg.num_classes).to(device)

trained_models = TrainedModels(
    rssm,
    encoder,
    decoder,
    reward_model,
    discount_model,
    actor,
    critic
)

In [ ]:
# 結果を動画で観てみるための関数
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML


def display_video(frames):
    plt.figure(figsize=(8, 8), dpi=50)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])
        plt.title("Step %d" % (i))

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
    display(HTML(anim.to_jshtml(default_mode='once')))
    plt.close()

In [ ]:
env = make_env(seed=1234)

policy = Agent(encoder, rssm, actor)

obs = env.reset()
done = False
total_reward = 0
frames = [obs]
actions = []

while not done:
    action = policy(obs)
    action_int = np.argmax(action)  # 環境に渡すときはint型

    obs, reward, done, _ = env.step(action_int)

    total_reward += reward
    frames.append(obs)
    actions.append(action_int)

print('Total Reward:', total_reward)

In [ ]:
display_video(frames)

In [ ]:
# repeat actionに対応した行動に変換する
submission_actions = np.zeros(len(actions) * env._skip)
for start_idx in range(env._skip):
    submission_actions[start_idx::env._skip] = np.array(actions)

np.save("drive/MyDrive/深層強化学習/最終課題/model", submission_actions)